Extracting User's Google Calendar

In [1]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery
import datetime


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import methods #file containing functions to interact with Google calendar like add event, create event, etc
import json
import re

google_calendar = methods.get_google_calendar_service() #get the google calendar service
tasks = methods.get_task_service() #get the task service
tasks_list = methods.extractTasks(tasks)
calendar = methods.extractCalendar(google_calendar) #extract the calendar from the service
client = methods.set_up_ChatGPT(calendar, tasks_list) #set up the ChatGPT model
now = methods.current_time()

Here are the events and tasks for the week:

Events:
1. 📅 545 Project Meeting on 2024-04-15 from 4:00 PM to 6:00 PM
2. 🏓 Pickle ball on 2024-04-15 from 6:00 PM to 7:00 PM
3. 😚😋😋 Doms on 2024-04-15 from 7:30 PM to 9:00 PM
4. Trivia @unionrec on 2024-04-16 from 7:00 PM to 9:00 PM
5. Dinner on 2024-04-17 from 6:00 PM to 7:00 PM
6. Jog on 2024-04-18 from 9:00 AM to 10:00 AM
7. 598 Project and Study on 2024-04-18 from 7:00 PM to 9:00 PM
8. Cooking on 2024-04-19 from 6:00 PM to 8:00 PM

Tasks:
- No tasks scheduled for the week

Logging Previous Conversation

In [4]:
def log_message(role, message):
    filename = f"conversation_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

In [5]:
def get_past_conversation():
    filename = f"conversation_log.txt"
    try:
        with open(filename, "r") as file:
            return file.read()
    except FileNotFoundError:
        return ""

Get User's Query

In [6]:
past_conversation = get_past_conversation()
# schedule tasks or calendar events

context = f"Here's my calendar and tasks for the week, store it for your reference.{calendar}\n{tasks_list} Today's date and time is: {now}\n"
instruction = """\nAs my virtual assistant, you are an expert with managing my calendar and tasks.
You can create, move, check, list, update, delete, and handle recurring events associated with my calendar based on the user's requests.
You can also manage my tasks by prioritizing them based on deadlines and scheduling them by creating an event.
When the user doesn't provide enough information, you can ask for more details, upto 3 times. Otherwise use your best judgement.
When the user wants to schedule their tasks, use your judgement for when tasks should be scheduled based on their deadline provided. Schedule a task as an event.
When the user wants to create an event or move an event and if the user hasn't specified when they want it scheduled, then refer to the calendar and use your best judgement to schedule the event. Keep in mind when the user would normally eat or sleep. Assume default duration for any event as 1 hour.
ALWAYS check for any conflicts before scheduling an event and notify the user if they are scheduling something that conflicts with another event.
Always ask for confirmation before making any changes to the calendar, if the user doesn't agree, then generate another suitable time slot.
When you are done, please let the user know that the task has been completed, and don't ask any more questions. If you aren't done, end with a question mark.
Please perform the necessary actions using the Google Calendar API. Here's the previous conversation with the user, read it and understand user preferences if any, use it to make your best judgement on scheduling.\n"""
print('calendar: ', calendar)
# initial setup
initial_prompt = context + instruction + past_conversation

calendar:  {'name': '545 Project Meeting', 'datetime_start': '2024-04-15T16:00:00-04:00', 'datetime_end': '2024-04-15T18:00:00-04:00', 'id': '33pmlrcvmobnp4oc9t4d64ib2o_20240415T200000Z'}
{'name': 'pickle ball🏓', 'datetime_start': '2024-04-15T18:00:00-04:00', 'datetime_end': '2024-04-15T19:00:00-04:00', 'id': 'ktfa5lvsrlot6b9vrbrj10kbk4'}
{'name': 'doms😚😋😋', 'datetime_start': '2024-04-15T19:30:00-04:00', 'datetime_end': '2024-04-15T21:00:00-04:00', 'id': 'ogmf7qgagmj778a951k8dalrgk'}
{'name': 'trivia @unionrec', 'datetime_start': '2024-04-16T19:00:00-04:00', 'datetime_end': '2024-04-16T21:00:00-04:00', 'id': '6mofg02ph0irf5vu39gdlhcltk'}
{'name': 'dinner', 'datetime_start': '2024-04-17T18:00:00-04:00', 'datetime_end': '2024-04-17T19:00:00-04:00', 'id': '526lbe7gfm8f0rltfrimpd7hsj'}
{'name': 'Jog', 'datetime_start': '2024-04-18T09:00:00-04:00', 'datetime_end': '2024-04-18T10:00:00-04:00', 'id': '9801kc07c3plj6cklpjspt80i0'}
{'name': '598 Project and Study', 'datetime_start': '2024-04-18

In [7]:
user_request = input()
combined_prompt = initial_prompt + "\nUser request: " + user_request


In [8]:
def chat_gpt(client, messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=500
    )
    return response.choices[0].message.content

messages = [
    {
        "role": "system",
        "content": initial_prompt
    },
    {
        "role": "user",
        "content": user_request
    }
]

log_message("user", user_request)

In [9]:
finalized_response = False

#establishing feedback loop to chat with user
while not finalized_response:
    response = chat_gpt(client, messages)
    print(response)
    log_message("assistant", response)
    messages.append({
        "role": "assistant",
        "content": response
    })
    if re.compile(r'\?').search(response):
        finalized_response = False
        user_feedback = input()
        messages.append({
            "role": "user",
            "content": user_feedback
        })
        log_message("user", user_feedback)
    else:
        finalized_response = True


I see that you have your "598 Project and Study" session currently scheduled for Thursday night at 7:00 PM. Would you like me to reschedule this session to tomorrow at 3:00 PM instead?
Great! I will reschedule your "598 Project and Study" session to tomorrow at 3:00 PM.

Your session has been successfully moved accordingly.

Is there anything else you would like me to assist with?
Great! If you need any further assistance, feel free to ask. Have a fantastic day!


In [18]:
#get GPT to output the function calls I need to make to the calendar API
instruction2 = """\nBased on the user's request you've just processed, please output the appropriate function call(s) to interact with the Google Calendar API. Format your response as a single string containing one or more function calls separated by semicolons. Do not include explanations or additional text.
All dates should be in 'YYYY-MM-DD' format, and times should be in 24-hour format 'HH:MM'. Additionally, for functions requiring a datetime range (such as 'checkSchedule' and 'ListEvents'), the datetime should be in ISO 8601 format 'YYYY-MM-DDTHH:MM:SSZ'.
Available function templates to include in the response string:
- createEvent(description, date:default=today, time, duration:default=1)
- moveEvent(event_id, new_date, new_time)
- updateEvent(event_id, new_description, new_duration, new_location)
- deleteEvent(event_id)
- createRecurringEvent(description, date, time, duration, frequency, until)
- deleteRecurringEvent(event_id)
- updateRecurringEvent(event_id, new_description, new_duration, new_location)

Make sure to fill in the required information, such as event names, IDs, dates, times, etc., based on the user's request context. Replace placeholders like 'today' or 'tomorrow' with specific dates in the format YYYY-MM-DD. If more details are needed from the user for a function call, use your best judgment to complete the function with the information available.
Please also make sure the event_id is for the corresponding event we are working with.
Make sure to actually fill out the event_id, not just write <event_id>
Example input: 'Schedule a team meeting for Friday afternoon at 3pm'
Example output: 'createEvent("Team Meeting", "YYYY-MM-DD", "15:00", 1)'
Example input: 'Update my team meeting to be two hours'
Example output 'updateEvent(<event_id>, '', 2, '')
Example input: 'Move my team meeting to tomorrow at 12pm'
Example output (Assuming today's date is 2024-15-04) 'moveEvent(<event_id>, '2024-16-04', '12:00')

Now, please generate the necessary function call(s) based on the user's request."""

In [19]:
messages.append({
    "role": "system",
    "content": instruction2
})
function_call = chat_gpt(client, messages)
print(function_call)
#Example of function call: createEvent("Meeting with John", "2022-12-31", "10:00 AM", 2)

moveEvent(6mofg02ph0irf5vu39gdlhcltk, 2024-04-16, 15:00)


In [20]:
function_calls = [line.strip() for line in function_call.split(';')]
print(function_calls)
print(calendar)

# Define a dictionary mapping function names to their corresponding Python functions
functions = {
    "createEvent": methods.createEvent,
    "moveEvent": methods.moveEvent,
    "checkSchedule": methods.checkSchedule,
    "updateEvent": methods.updateEvent,
}

#TODO: works for createEvent, gotta change other functions in methods file to accept arguments as strings and convert them to appropriate data types
# Iterate through the function calls
for call in function_calls:
    # Split the string to extract function name and arguments
    if call != '':
        parts = call.split('(')
        function_name = parts[0]
        arguments = parts[1].rstrip(')').split(', ')

    # Call the corresponding function with the provided arguments
    if function_name in functions:
        functions[function_name](*arguments)
    else:
        print(f"Function '{function_name}' not found.")

['moveEvent(6mofg02ph0irf5vu39gdlhcltk, 2024-04-16, 15:00)']
{'name': '545 Project Meeting', 'datetime_start': '2024-04-15T16:00:00-04:00', 'datetime_end': '2024-04-15T18:00:00-04:00', 'id': '33pmlrcvmobnp4oc9t4d64ib2o_20240415T200000Z'}
{'name': 'pickle ball🏓', 'datetime_start': '2024-04-15T18:00:00-04:00', 'datetime_end': '2024-04-15T19:00:00-04:00', 'id': 'ktfa5lvsrlot6b9vrbrj10kbk4'}
{'name': 'doms😚😋😋', 'datetime_start': '2024-04-15T19:30:00-04:00', 'datetime_end': '2024-04-15T21:00:00-04:00', 'id': 'ogmf7qgagmj778a951k8dalrgk'}
{'name': 'trivia @unionrec', 'datetime_start': '2024-04-16T19:00:00-04:00', 'datetime_end': '2024-04-16T21:00:00-04:00', 'id': '6mofg02ph0irf5vu39gdlhcltk'}
{'name': 'dinner', 'datetime_start': '2024-04-17T18:00:00-04:00', 'datetime_end': '2024-04-17T19:00:00-04:00', 'id': '526lbe7gfm8f0rltfrimpd7hsj'}
{'name': 'Jog', 'datetime_start': '2024-04-18T09:00:00-04:00', 'datetime_end': '2024-04-18T10:00:00-04:00', 'id': '9801kc07c3plj6cklpjspt80i0'}
{'name': '598